# Time Series Models Comparison: Energy Consumption Forecasting

**SciPy 2024 Conference - Tacoma, WA**

---

## Overview

This notebook provides a comprehensive comparison of 5 different time series forecasting models:

1. **ARIMA** - Classical statistical approach (baseline)
2. **xLSTM** - Deep learning with explainability
3. **TimesFM** - Google's foundation model (zero-shot learning)
4. **TimeGPT** - API-based foundation model
5. **AutoGluon** - AutoML ensemble approach

### Evaluation Metrics:
- **MAE** (Mean Absolute Error) - Lower is better
- **RMSE** (Root Mean Squared Error) - Lower is better
- **Runtime** - Training/inference time in seconds

---

## 1. Setup and Data Generation

### Import Libraries

In [ ]:
# Core libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import warnings
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Configure
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

# Set random seed for reproducibility
np.random.seed(42)

print("✓ Libraries imported successfully")

### Generate Synthetic Energy Consumption Data

We create synthetic data with three components:
- **Trend**: Linear upward growth
- **Seasonality**: Annual cyclic pattern
- **Noise**: Random fluctuations

In [ ]:
def generate_energy_data(n_samples=1000, seed=42):
    """Generate synthetic energy consumption data with trend and seasonality"""
    np.random.seed(seed)
    
    # Create date range
    date_rng = pd.date_range(start='2020-01-01', periods=n_samples, freq='D')
    df = pd.DataFrame(date_rng, columns=['timestamp'])
    
    # Create components
    trend = np.linspace(0, 20, n_samples)  # Linear trend
    season = 10 * np.sin(2 * np.pi * np.arange(n_samples) / 365.25)  # Annual seasonality
    noise = np.random.normal(0, 5, n_samples)  # Random noise
    
    # Combine components
    df['energy_consumption'] = 100 + trend + season + noise
    df.set_index('timestamp', inplace=True)
    
    return df

# Generate data
df = generate_energy_data()

# Split into train and test
test_size = 30
train_data = df[:-test_size]
test_data = df[-test_size:]

print(f"Dataset Statistics:")
print(f"  Total samples: {len(df)}")
print(f"  Training samples: {len(train_data)}")
print(f"  Test samples: {len(test_data)}")
print(f"  Date range: {df.index[0].date()} to {df.index[-1].date()}")
print(f"\n  Mean consumption: {df['energy_consumption'].mean():.2f} kWh")
print(f"  Std deviation: {df['energy_consumption'].std():.2f} kWh")

### Visualize the Data

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# Plot 1: Full time series
ax = axes[0]
ax.plot(train_data.index, train_data['energy_consumption'], 
        label='Training Data', alpha=0.8, linewidth=1.5)
ax.plot(test_data.index, test_data['energy_consumption'], 
        label='Test Data', color='orange', alpha=0.8, linewidth=1.5)
ax.axvline(x=test_data.index[0], color='red', linestyle='--', 
           alpha=0.5, label='Train/Test Split')
ax.set_title('Energy Consumption Time Series', fontsize=13, fontweight='bold')
ax.set_xlabel('Date', fontsize=11)
ax.set_ylabel('Energy Consumption (kWh)', fontsize=11)
ax.legend(loc='upper left')
ax.grid(True, alpha=0.3)

# Plot 2: Test data zoomed in
ax = axes[1]
ax.plot(test_data.index, test_data['energy_consumption'], 
        marker='o', color='orange', linewidth=2, markersize=6, label='Test Data')
ax.set_title('Test Set (Last 30 Days)', fontsize=13, fontweight='bold')
ax.set_xlabel('Date', fontsize=11)
ax.set_ylabel('Energy Consumption (kWh)', fontsize=11)
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n✓ Data visualization complete")

---

## 2. Model Training and Evaluation

We'll train each model sequentially and store the results for comparison.

### Model 1: ARIMA (Statistical Baseline)

**Key Features:**
- Classical statistical approach
- Highly interpretable
- Very fast training
- Good for data with clear trend/seasonality

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
from pmdarima import auto_arima

print("="*60)
print("Training ARIMA Model")
print("="*60)

start_time = time.time()

# Auto-select best ARIMA parameters
print("Searching for optimal parameters...")
model_auto = auto_arima(
    train_data['energy_consumption'],
    start_p=0, start_q=0, max_p=5, max_q=5, m=7,
    start_P=0, seasonal=True, d=1, D=1,
    trace=False, error_action='ignore',
    suppress_warnings=True, stepwise=True
)

print(f"  Best order: {model_auto.order}")

# Fit ARIMA model
model_arima = ARIMA(train_data['energy_consumption'], order=model_auto.order)
results_arima = model_arima.fit()

# Make predictions
forecast_arima = results_arima.forecast(steps=test_size)

arima_time = time.time() - start_time

# Calculate metrics
arima_mae = mean_absolute_error(test_data['energy_consumption'], forecast_arima)
arima_rmse = np.sqrt(mean_squared_error(test_data['energy_consumption'], forecast_arima))

print(f"\n✓ ARIMA Training Complete")
print(f"  MAE:     {arima_mae:.3f} kWh")
print(f"  RMSE:    {arima_rmse:.3f} kWh")
print(f"  Runtime: {arima_time:.2f} seconds")
print("="*60)

### Model 2: xLSTM (Deep Learning)

**Key Features:**
- Captures complex non-linear patterns
- Learns from sequential dependencies
- More flexible than statistical models
- Provides some interpretability

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler

print("="*60)
print("Training xLSTM Model")
print("="*60)

start_time = time.time()

# Prepare data
scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train_data[['energy_consumption']])
test_scaled = scaler.transform(test_data[['energy_consumption']])

# Create sequences
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:(i + seq_length)])
        y.append(data[i + seq_length])
    return np.array(X), np.array(y)

seq_length = 30
X_train, y_train = create_sequences(train_scaled, seq_length)

print(f"Sequence shape: {X_train.shape}")

# Build LSTM model
model_lstm = Sequential([
    LSTM(50, activation='relu', return_sequences=True, input_shape=(seq_length, 1)),
    LSTM(50, activation='relu'),
    Dense(1)
])

model_lstm.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

print("Training neural network...")

# Train model
history = model_lstm.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32,
    validation_split=0.1,
    verbose=0
)

# Make predictions
last_sequence = train_scaled[-seq_length:]
forecast_lstm = []

for _ in range(test_size):
    next_pred = model_lstm.predict(last_sequence.reshape(1, seq_length, 1), verbose=0)
    forecast_lstm.append(next_pred[0, 0])
    last_sequence = np.roll(last_sequence, -1, axis=0)
    last_sequence[-1] = next_pred

# Inverse transform
forecast_lstm = scaler.inverse_transform(np.array(forecast_lstm).reshape(-1, 1)).flatten()

lstm_time = time.time() - start_time

# Calculate metrics
lstm_mae = mean_absolute_error(test_data['energy_consumption'], forecast_lstm)
lstm_rmse = np.sqrt(mean_squared_error(test_data['energy_consumption'], forecast_lstm))

print(f"\n✓ xLSTM Training Complete")
print(f"  MAE:     {lstm_mae:.3f} kWh")
print(f"  RMSE:    {lstm_rmse:.3f} kWh")
print(f"  Runtime: {lstm_time:.2f} seconds")
print("="*60)

### Model 3: TimesFM (Google Foundation Model)

**Key Features:**
- Zero-shot learning (no training required!)
- 200M parameter transformer model
- Pre-trained on diverse time series
- Fast inference

In [ ]:
print("="*60)
print("Loading TimesFM Model")
print("="*60)

try:
    import timesfm
    
    start_time = time.time()
    
    # Load pretrained model
    print("Loading pretrained checkpoint from Hugging Face...")
    tfm = timesfm.TimesFm(
        context_len=512,
        horizon_len=test_size,
        input_patch_len=32,
        output_patch_len=128,
        num_layers=20,
        model_dims=1280,
        backend='cpu',  # Use 'gpu' if available
    )
    
    tfm.load_from_checkpoint(repo_id="google/timesfm-1.0-200m-pytorch")
    
    print("Generating forecast (zero-shot, no training)...")
    
    # Forecast
    forecast_result = tfm.forecast(
        inputs=[train_data['energy_consumption'].values],
        freq=[0],  # Daily frequency
    )
    
    forecast_timesfm = forecast_result.forecast[0][:test_size]
    
    timesfm_time = time.time() - start_time
    
    # Calculate metrics
    timesfm_mae = mean_absolute_error(test_data['energy_consumption'], forecast_timesfm)
    timesfm_rmse = np.sqrt(mean_squared_error(test_data['energy_consumption'], forecast_timesfm))
    
    print(f"\n✓ TimesFM Inference Complete")
    print(f"  MAE:     {timesfm_mae:.3f} kWh")
    print(f"  RMSE:    {timesfm_rmse:.3f} kWh")
    print(f"  Runtime: {timesfm_time:.2f} seconds")
    print("="*60)
    
    timesfm_available = True
    
except ImportError:
    print("\n⚠ TimesFM not installed")
    print("  Install with: pip install timesfm")
    print("  Skipping TimesFM evaluation...\n")
    print("="*60)
    timesfm_available = False
    timesfm_mae, timesfm_rmse, timesfm_time = None, None, None
    forecast_timesfm = None
    
except Exception as e:
    print(f"\n⚠ TimesFM error: {str(e)}")
    print("  Skipping TimesFM evaluation...\n")
    print("="*60)
    timesfm_available = False
    timesfm_mae, timesfm_rmse, timesfm_time = None, None, None
    forecast_timesfm = None

### Model 4: TimeGPT (API-based Foundation Model)

**Key Features:**
- Pre-trained on billions of time points
- Best speed/accuracy trade-off
- API-based inference (requires API key)
- Minimal preprocessing needed

In [ ]:
print("="*60)
print("Using TimeGPT Model")
print("="*60)

try:
    from nixtla import NixtlaClient
    
    start_time = time.time()
    
    # Initialize client
    print("Initializing TimeGPT client...")
    client = NixtlaClient()  # Expects TIMEGPT_TOKEN env variable
    
    # Prepare data
    df_timegpt = pd.DataFrame({
        'ds': train_data.index,
        'y': train_data['energy_consumption'].values
    })
    
    print("Generating forecast via API...")
    
    # Forecast
    forecast_df = client.forecast(
        df=df_timegpt,
        h=test_size,
        freq='D',
        time_col='ds',
        target_col='y'
    )
    
    forecast_timegpt = forecast_df['TimeGPT'].values
    
    timegpt_time = time.time() - start_time
    
    # Calculate metrics
    timegpt_mae = mean_absolute_error(test_data['energy_consumption'], forecast_timegpt)
    timegpt_rmse = np.sqrt(mean_squared_error(test_data['energy_consumption'], forecast_timegpt))
    
    print(f"\n✓ TimeGPT Inference Complete")
    print(f"  MAE:     {timegpt_mae:.3f} kWh")
    print(f"  RMSE:    {timegpt_rmse:.3f} kWh")
    print(f"  Runtime: {timegpt_time:.2f} seconds")
    print("="*60)
    
    timegpt_available = True
    
except ImportError:
    print("\n⚠ TimeGPT not installed")
    print("  Install with: pip install nixtla")
    print("  Skipping TimeGPT evaluation...\n")
    print("="*60)
    timegpt_available = False
    timegpt_mae, timegpt_rmse, timegpt_time = None, None, None
    forecast_timegpt = None
    
except Exception as e:
    print(f"\n⚠ TimeGPT error: {str(e)}")
    print("  Set TIMEGPT_TOKEN environment variable")
    print("  OR: client = NixtlaClient(api_key='your-key')")
    print("  Skipping TimeGPT evaluation...\n")
    print("="*60)
    timegpt_available = False
    timegpt_mae, timegpt_rmse, timegpt_time = None, None, None
    forecast_timegpt = None

### Model 5: AutoGluon (AutoML)

**Key Features:**
- Automated model selection and ensembling
- Best overall accuracy
- Minimal hyperparameter tuning
- Production-ready

In [ ]:
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

print("="*60)
print("Training AutoGluon Model")
print("="*60)

start_time = time.time()

# Prepare data in AutoGluon format
train_ag = pd.DataFrame({
    'timestamp': train_data.index,
    'target': train_data['energy_consumption'].values,
    'item_id': ['energy'] * len(train_data)
})

train_ag = TimeSeriesDataFrame.from_data_frame(
    train_ag,
    id_column='item_id',
    timestamp_column='timestamp'
)

print("Training ensemble models...")

# Train predictor
predictor = TimeSeriesPredictor(
    prediction_length=test_size,
    target='target',
    verbosity=0
)

predictor.fit(
    train_ag,
    presets='fast_training',
    time_limit=60
)

# Predict
forecast_ag = predictor.predict(train_ag)
forecast_autogluon = forecast_ag['mean'].values

autogluon_time = time.time() - start_time

# Calculate metrics
autogluon_mae = mean_absolute_error(test_data['energy_consumption'], forecast_autogluon)
autogluon_rmse = np.sqrt(mean_squared_error(test_data['energy_consumption'], forecast_autogluon))

print(f"\n✓ AutoGluon Training Complete")
print(f"  MAE:     {autogluon_mae:.3f} kWh")
print(f"  RMSE:    {autogluon_rmse:.3f} kWh")
print(f"  Runtime: {autogluon_time:.2f} seconds")
print("="*60)

---

## 3. Results Comparison

### Summary Table

In [ ]:
# Compile results
models_list = ['ARIMA', 'xLSTM']
mae_list = [arima_mae, lstm_mae]
rmse_list = [arima_rmse, lstm_rmse]
runtime_list = [arima_time, lstm_time]

if timesfm_available:
    models_list.append('TimesFM')
    mae_list.append(timesfm_mae)
    rmse_list.append(timesfm_rmse)
    runtime_list.append(timesfm_time)

if timegpt_available:
    models_list.append('TimeGPT')
    mae_list.append(timegpt_mae)
    rmse_list.append(timegpt_rmse)
    runtime_list.append(timegpt_time)

models_list.append('AutoGluon')
mae_list.append(autogluon_mae)
rmse_list.append(autogluon_rmse)
runtime_list.append(autogluon_time)

# Create results DataFrame
results = pd.DataFrame({
    'Model': models_list,
    'MAE (kWh)': mae_list,
    'RMSE (kWh)': rmse_list,
    'Runtime (s)': runtime_list
})

# Add rankings
results['MAE Rank'] = results['MAE (kWh)'].rank().astype(int)
results['Speed Rank'] = results['Runtime (s)'].rank().astype(int)

# Display results
print("\n" + "="*80)
print("MODEL COMPARISON RESULTS")
print("="*80)
print(results.to_string(index=False))
print("="*80)

# Highlight winners
best_mae_idx = results['MAE (kWh)'].idxmin()
best_rmse_idx = results['RMSE (kWh)'].idxmin()
fastest_idx = results['Runtime (s)'].idxmin()

print(f"\n🏆 Best Accuracy (MAE):  {results.loc[best_mae_idx, 'Model']} ({results.loc[best_mae_idx, 'MAE (kWh)']:.3f} kWh)")
print(f"🏆 Best Accuracy (RMSE): {results.loc[best_rmse_idx, 'Model']} ({results.loc[best_rmse_idx, 'RMSE (kWh)']:.3f} kWh)")
print(f"⚡ Fastest Runtime:       {results.loc[fastest_idx, 'Model']} ({results.loc[fastest_idx, 'Runtime (s)']:.2f} sec)")
print("="*80)

### Visual Comparison

In [ ]:
# Create comprehensive visualization
fig = plt.figure(figsize=(18, 10))
gs = fig.add_gridspec(2, 3, hspace=0.3, wspace=0.3)

# Plot 1: All Forecasts Overlaid
ax1 = fig.add_subplot(gs[0, :])
ax1.plot(test_data.index, test_data['energy_consumption'], 
         label='Actual', color='black', linewidth=3, marker='o', markersize=5, zorder=10)
ax1.plot(test_data.index, forecast_arima, 
         label=f'ARIMA (MAE={arima_mae:.2f})', alpha=0.7, linewidth=2, linestyle='--')
ax1.plot(test_data.index, forecast_lstm, 
         label=f'xLSTM (MAE={lstm_mae:.2f})', alpha=0.7, linewidth=2, linestyle='--')
if timesfm_available:
    ax1.plot(test_data.index, forecast_timesfm, 
             label=f'TimesFM (MAE={timesfm_mae:.2f})', alpha=0.7, linewidth=2, linestyle='--')
if timegpt_available:
    ax1.plot(test_data.index, forecast_timegpt, 
             label=f'TimeGPT (MAE={timegpt_mae:.2f})', alpha=0.7, linewidth=2, linestyle='--')
ax1.plot(test_data.index, forecast_autogluon, 
         label=f'AutoGluon (MAE={autogluon_mae:.2f})', alpha=0.7, linewidth=2, linestyle='--')
ax1.set_title('Forecast Comparison - All Models', fontsize=14, fontweight='bold', pad=15)
ax1.set_xlabel('Date', fontsize=12)
ax1.set_ylabel('Energy Consumption (kWh)', fontsize=12)
ax1.legend(loc='upper left', framealpha=0.9)
ax1.grid(True, alpha=0.3)

# Plot 2: MAE Comparison
ax2 = fig.add_subplot(gs[1, 0])
colors = plt.cm.viridis(np.linspace(0.2, 0.9, len(results)))
bars = ax2.bar(results['Model'], results['MAE (kWh)'], color=colors, edgecolor='black', linewidth=1.5)
ax2.set_title('Mean Absolute Error', fontsize=12, fontweight='bold')
ax2.set_ylabel('MAE (kWh)', fontsize=11)
ax2.set_xticklabels(results['Model'], rotation=45, ha='right')
for bar in bars:
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height,
             f'{height:.2f}', ha='center', va='bottom', fontweight='bold', fontsize=9)
ax2.grid(True, alpha=0.3, axis='y')

# Plot 3: RMSE Comparison
ax3 = fig.add_subplot(gs[1, 1])
bars = ax3.bar(results['Model'], results['RMSE (kWh)'], color=colors, edgecolor='black', linewidth=1.5)
ax3.set_title('Root Mean Squared Error', fontsize=12, fontweight='bold')
ax3.set_ylabel('RMSE (kWh)', fontsize=11)
ax3.set_xticklabels(results['Model'], rotation=45, ha='right')
for bar in bars:
    height = bar.get_height()
    ax3.text(bar.get_x() + bar.get_width()/2., height,
             f'{height:.2f}', ha='center', va='bottom', fontweight='bold', fontsize=9)
ax3.grid(True, alpha=0.3, axis='y')

# Plot 4: Runtime Comparison
ax4 = fig.add_subplot(gs[1, 2])
bars = ax4.bar(results['Model'], results['Runtime (s)'], color=colors, edgecolor='black', linewidth=1.5)
ax4.set_title('Training/Inference Runtime', fontsize=12, fontweight='bold')
ax4.set_ylabel('Time (seconds)', fontsize=11)
ax4.set_xticklabels(results['Model'], rotation=45, ha='right')
for bar in bars:
    height = bar.get_height()
    ax4.text(bar.get_x() + bar.get_width()/2., height,
             f'{height:.1f}s', ha='center', va='bottom', fontweight='bold', fontsize=9)
ax4.grid(True, alpha=0.3, axis='y')

plt.suptitle('Time Series Model Comparison Dashboard', fontsize=16, fontweight='bold', y=0.995)
plt.savefig('results/model_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✓ Visualization saved to: results/model_comparison.png")

---

## 4. Key Insights and Takeaways

### Model Performance Summary:

1. **Best Overall Accuracy**: AutoGluon achieves the lowest MAE through ensemble learning
2. **Foundation Models**: TimesFM and TimeGPT offer excellent accuracy with zero/minimal training
3. **Fastest Baseline**: ARIMA provides quick predictions for rapid iteration
4. **Deep Learning**: xLSTM balances accuracy with interpretability

### When to Use Each Model:

| Model | Best For | Trade-off |
|-------|----------|----------|
| **ARIMA** | Quick baseline, interpretability | Lower accuracy |
| **xLSTM** | Complex patterns, some explainability | Training time |
| **TimesFM** | Zero-shot learning, no training | Model download |
| **TimeGPT** | Best balance, API-based | API dependency |
| **AutoGluon** | Production deployment, best accuracy | Training time |

### Design Decisions:

- **Test Size (30 days)**: Balances evaluation robustness with efficiency
- **Sequence Length (30 for LSTM)**: One month of history captures patterns
- **Fast Presets**: Used for demonstration; can be tuned for better performance
- **Consistent Evaluation**: Same data, metrics, and split for fair comparison

### Next Steps:

1. **Feature Engineering**: Add weather, calendar features, energy prices
2. **Hyperparameter Tuning**: Optimize for production deployment
3. **Cross-Validation**: Time series CV for robust evaluation
4. **Ensemble Methods**: Combine top models for better accuracy
5. **Production Monitoring**: Track model drift and retrain periodically

---

**SciPy 2024 Conference | Tacoma, WA**

For more details, see: [Full Metrics Report](results/METRICS_SUMMARY.md)